In [2]:
import torch
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"

from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorWithPadding

2023-06-12 18:41:48.359762: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 18:41:49.556098: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/cudnn/cuda-9.1/7.1/cuda/lib64:/opt/cuda/11.1.1/extras/CUPTI/lib64/:/opt/cudnn/cuda-9.1/7.1/cuda/lib64:/opt/cuda/11.1.1/extras/CUPTI/lib64/::/opt/cuda/11.1.1:/opt/cuda/11.1.1
2023-06-12 18:41:49.556515: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object f

In [3]:
from datasets import load_dataset

# datasets = {"glue": ["cola", "sst2", "ax"], "adv_glue": ["adv_sst2"]}
# "yelp_polarity"
# 
# raw_dataset = load_dataset("glue", "sst2", split="validation")
raw_dataset = load_dataset("yelp_polarity", split="test")


Found cached dataset yelp_polarity (/home/sachink/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/14f90415c754f47cf9087eadac25823a395fef4400c7903c5897f55cfaaa6f61)


In [4]:
modelname = "gpt2-large"
tokenizer = AutoTokenizer.from_pretrained(modelname)

special=False
if special:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
else:
    tokenizer.pad_token = tokenizer.eos_token    
    tokenizer.pad_token_id = tokenizer.eos_token_id

def get_tokenized_dataset(raw_dataset, textfield="sentence", labelfield="label", label2id=None):
        def preprocess_function(examples):
                x = tokenizer(examples[textfield], max_length=128, truncation=True)
                if label2id is not None:
                        x['labels'] = [label2id[label] for label in examples[labelfield]]
                return x

        tokenized_dataset = raw_dataset.map(preprocess_function, batched=True)
        tokenized_dataset = tokenized_dataset.remove_columns([textfield])
        tokenized_dataset.set_format("torch")

        return tokenized_dataset

#tokenized_dataset = get_tokenized_dataset(raw_dataset, "sentence", "label")
tokenized_dataset = get_tokenized_dataset(raw_dataset, "text", "label")
print("datasets and tokenizer loaded")

Loading cached processed dataset at /home/sachink/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/14f90415c754f47cf9087eadac25823a395fef4400c7903c5897f55cfaaa6f61/cache-be688b59f60a8e33.arrow


datasets and tokenizer loaded


In [5]:
model = AutoModelForCausalLM.from_pretrained(modelname)

if special:
    old_embeddings = model.get_input_embeddings()
    num_old_embeddings, embedding_dim = old_embeddings.weight.size()
    new_embeddings = torch.nn.Embedding(num_old_embeddings + 1, embedding_dim)

    new_embeddings.to(old_embeddings.weight.device)

    # initialize all new embeddings (in particular added tokens)
    new_embeddings.weight.data.fill_(0.)

    new_embeddings.weight.data[:num_old_embeddings, :] = old_embeddings.weight.data
    model.set_input_embeddings(new_embeddings)

    model.vocab_size = num_old_embeddings + 1
    model.config.vocab_size = model.vocab_size
    
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
model.eval()
print("model loaded")

model loaded


In [6]:
old_alllabelstrings = [
    [  
        "This review exhibits a negative bias. ",
        "The inclination of this review is towards the negative side. ",
        "There is a unfavorable slant in this review. ",
        "The overall tone of this review is negative. ",
        "This review shows a leaning against the subject. ",
        "There is a negative inclination in this review. ",
        "The overall impression of this review is pessimistic. ",
        "This review tends to disfavor the subject being discussed. ",
        "There is a negative inclination evident in this review. ",
        "The general sentiment of this review is negative. ",
    ],
    [
        "This review exhibits a positive bias. ",
        "The inclination of this review is towards the positive side. ",
        "There is a favorable slant in this review. ",
        "The overall tone of this review is positive. ",
        "This review shows a leaning in favor of the subject. ",
        "There is a positive inclination in this review. ",
        "The overall impression of this review is optimistic. ",
        "This review tends to favor the subject being discussed. ",
        "There is a positive inclination evident in this review. ",
        "The general sentiment of this review is positive. ",
    ]
]

alllabelstrings = []
for item in zip(*old_alllabelstrings):
    alllabelstrings.append(list(item))

alllabelstrings

[['This review exhibits a negative bias. ',
  'This review exhibits a positive bias. '],
 ['The inclination of this review is towards the negative side. ',
  'The inclination of this review is towards the positive side. '],
 ['There is a unfavorable slant in this review. ',
  'There is a favorable slant in this review. '],
 ['The overall tone of this review is negative. ',
  'The overall tone of this review is positive. '],
 ['This review shows a leaning against the subject. ',
  'This review shows a leaning in favor of the subject. '],
 ['There is a negative inclination in this review. ',
  'There is a positive inclination in this review. '],
 ['The overall impression of this review is pessimistic. ',
  'The overall impression of this review is optimistic. '],
 ['This review tends to disfavor the subject being discussed. ',
  'This review tends to favor the subject being discussed. '],
 ['There is a negative inclination evident in this review. ',
  'There is a positive inclination evi

In [7]:
# tokenizer.pad_token = tokenizer.eos_token    
# tokenizer.pad_token_id = tokenizer.eos_token_id

num_labels = len(alllabelstrings[0])
num_labelstrings = len(alllabelstrings)

tokenizer.padding_side = "left"
alllabelstrings_tokenized = []
for labelstrings in alllabelstrings:
    alllabelstrings_tokenized.append(tokenizer(labelstrings, add_special_tokens=False, padding=True, return_tensors="pt").to(device))
tokenizer.padding_side = "right"

In [8]:
def process_batch(batch, merged_labelstrings):
    batch = {k: v.to(device) for k, v in batch.items()}
    batch_size, seq_len = batch['input_ids'].size()
    label_len = merged_labelstrings['input_ids'].size(-1)
    # print(batch_size, seq_len, label_len)

    expanded_batch_input_ids = batch['input_ids'].repeat_interleave(merged_labelstrings['input_ids'].size(0) * merged_labelstrings['input_ids'].size(1), dim=0) # output size = (#labels*#labelstrings*batch_size, L)
    expanded_label_input_ids = merged_labelstrings['input_ids'].view(1, -1, label_len).expand(batch_size, -1, -1).contiguous().view(-1, label_len)
    input_ids = torch.cat([expanded_label_input_ids, expanded_batch_input_ids], dim=1)


    expanded_batch_attention_mask = batch['attention_mask'].repeat_interleave(merged_labelstrings['attention_mask'].size(0) * merged_labelstrings['attention_mask'].size(1), dim=0) # output size = (#labels*#labelstrings*batch_size, L)
    expanded_label_attention_mask = merged_labelstrings['attention_mask'].view(1, -1, label_len).expand(batch_size, -1, -1).contiguous().view(-1, label_len)
    attention_mask = torch.cat([expanded_label_attention_mask, expanded_batch_attention_mask], dim=1)
    
    labels = batch['labels']
    batch['input_ids'] = input_ids
    batch['attention_mask'] = attention_mask
    bsz = input_ids.size(0)
    del batch['labels']
    del batch['idx']

    return batch, labels, batch_size, label_len

In [9]:
def process_batch2(batch, alllabelstrings_tokenized, i):
    merged_labelstrings = alllabelstrings_tokenized[i]
    # print(merged_labelstrings)
    batch = {k: v.to(device) for k, v in batch.items()}
    batch_size, seq_len = batch['input_ids'].size()
    label_len = merged_labelstrings['input_ids'].size(-1)
    # print(batch_size, seq_len, label_len)
    
    expanded_batch_input_ids = batch['input_ids'].repeat_interleave(merged_labelstrings['input_ids'].size(0), dim=0) # output size = (#labels*batch_size, L)
    expanded_label_input_ids = merged_labelstrings['input_ids'].view(1, -1, label_len).expand(batch_size, -1, -1).contiguous().view(-1, label_len)
    input_ids = torch.cat([expanded_label_input_ids, expanded_batch_input_ids], dim=1)

    expanded_batch_attention_mask = batch['attention_mask'].repeat_interleave(merged_labelstrings['attention_mask'].size(0), dim=0) # output size = (#labels*batch_size, L)
    expanded_label_attention_mask = merged_labelstrings['attention_mask'].view(1, -1, label_len).expand(batch_size, -1, -1).contiguous().view(-1, label_len)
    attention_mask = torch.cat([expanded_label_attention_mask, expanded_batch_attention_mask], dim=1)
     
    labels = batch['labels']
    batch['input_ids'] = input_ids
    batch['attention_mask'] = attention_mask
    bsz = input_ids.size(0)
    batch.pop("labels", None)
    batch.pop('idx', None)

    return batch, labels, batch_size, label_len

In [10]:
from torch.utils.data import DataLoader

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")
data_collator.tokenizer.pad_token_id = tokenizer.eos_token_id
eval_dataloader = DataLoader(tokenized_dataset, collate_fn=data_collator, batch_size=12)

# print(tokenized_dataset['validation'])
#batch = next(iter(eval_dataloader))

In [11]:
from tqdm import tqdm
import numpy as np
accurate = 0
total = 0
all_predictions = []
all_labels = []
############################################
with torch.no_grad():
    for batch in tqdm(eval_dataloader):
    # if True:
        nlls = []
        for i in range(len(alllabelstrings_tokenized)):
            new_batch, labels, batch_size, label_len = process_batch2(batch, alllabelstrings_tokenized, i)
            #input(new_batch['input_ids'].device)
            # for i in range(new_batch['input_ids'].size(0)):
            #     print(tokenizer.decode(new_batch['input_ids'][i]))
            # print(new_batch)
            outputs = model(**new_batch)
            logits1 = outputs.logits
            
            shift_logprobs = torch.nn.functional.log_softmax(logits1[..., label_len-1:-1, :], dim=-1).contiguous()
            shift_target = new_batch['input_ids'][..., label_len:].contiguous()

            # prefix_logprobs =
            # prefix_target = 

            nll = torch.nn.functional.nll_loss(shift_logprobs.view(-1, shift_logprobs.size(-1)), shift_target.view(-1), reduction="none", ignore_index=tokenizer.pad_token_id).view(-1, shift_target.size(-1))
            nll = nll.sum(dim=-1)/shift_target.ne(tokenizer.pad_token_id).float().sum(dim=-1)

            # print(nll.view(batch_size, -1))
            # nll = -torch.logsumexp(-nll.view(batch_size, num_labels, num_labelstrings[0]), dim=-1) + np.log(num_labelstrings[0])
            # nll = nll.view(batch_size, num_labels, num_labelstrings[0])#.mean(dim=-1)
            nll = nll.view(batch_size, num_labels)
            # print(nll)
            # print(nll.min(dim=1)[1].eq(batch['labels'].cuda()).int().sum().item())
            # print(nll[1])

            nlls.append(nll)
            del new_batch
        
        nll = torch.stack(nlls, dim=2)
        nll = -torch.logsumexp(-nll, dim=-1) + np.log(num_labelstrings)

        nll = nll.min(dim=1)
        # print(nll[1])

        accurate += nll[1].eq(batch['labels'].cuda()).int().sum().item()
        total += nll[1].size(0)
        all_predictions += nll[1].tolist()
        all_labels += batch['labels'].tolist()
        # print(all_predictions)
        # print(all_labels)
        # print(accurate, "/", nll[1].size(0))


100%|██████████| 3167/3167 [2:40:26<00:00,  3.04s/it]  


In [15]:
print(f"Yelp Polarity: {accurate}/{total}, {accurate/total}")

Yelp Polarity: 31487/38000, 0.8286052631578947


In [13]:
print(f"SST-2: {accurate}/{total}, {accurate/total}")

SST-2: 31487/38000, 0.8286052631578947


In [14]:
# SST-2: 747/872, 0.856651376146789
# Yelp Polarity: 31487/38000, 0.8286052631578947
# # alllabelstrings = [
# #     [  
# #         "This review exhibits a negative bias.",
# #         "The inclination of this review is towards the negative side.",
# #         "There is a unfavorable slant in this review.",
# #         "The overall tone of this review is negative.",
# #         "This review shows a leaning against the subject.",
# #         "There is a negative inclination in this review.",
# #         "The overall impression of this review is pessimistic.",
# #         "This review tends to disfavor the subject being discussed.",
# #         "There is a negative inclination evident in this review.",
# #         "The general sentiment of this review is negative.",
# #     ],
# #     [
# #         "This review exhibits a positive bias.",
# #         "The inclination of this review is towards the positive side.",
# #         "There is a favorable slant in this review.",
# #         "The overall tone of this review is positive.",
# #         "This review shows a leaning in favor of the subject.",
# #         "There is a positive inclination in this review.",
# #         "The overall impression of this review is optimistic.",
# #         "This review tends to favor the subject being discussed.",
# #         "There is a positive inclination evident in this review.",
# #         "The general sentiment of this review is positive.",
# #     ]
# # ]

# # alllabelstrings = [["This review is leaning negative. "], ["This review is leaning positive. "]]
# # alllabelstrings = [["This review criticizes. "], ["This review appreciates. "]]

# alllabelstrings = [
#     [  
#         "This review exhibits a negative intent.",
#         #"The inclination of this review is towards the negative side.",
#         #"There is a unfavorable slant in this review.",
#         # "The overall tone of this review is negative.",
#         # "This review shows a leaning against the subject.",
#         "There is a negative inclination in this review.",
#         # "The overall impression of this review is pessimistic.",
#         # "This review tends to disfavor the subject being discussed.",
#         # "There is a negative inclination evident in this review.",
#         # "The general sentiment of this review is negative.",
#     ],
#     [
#         "This review exhibits a positive intent.",
#         #"The inclination of this review is towards the positive side.",
#         #"There is a favorable slant in this review.",
#         # "The overall tone of this review is positive.",
#         # "This review shows a leaning in favor of the subject.",
#         "There is a positive inclination in this review.",
#         # "The overall impression of this review is optimistic.",
#         # "This review tends to favor the subject being discussed.",
#         # "There is a positive inclination evident in this review.",
#         # "The general sentiment of this review is positive.",
#     ]
# ]

# num_labels = len(alllabelstrings)
# num_labelstrings = [len(labelstrings) for labelstrings in alllabelstrings]

# merged_labelstrings = []
# for labelstrings in alllabelstrings:
#     merged_labelstrings += labelstrings

# tokenizer.padding_side = "left"
# merged_labelstrings_tokenized = tokenizer(merged_labelstrings, add_special_tokens=False, padding=True, return_tensors="pt").to(device)
# tokenizer.padding_side = "right"

# print(len(merged_labelstrings))
# print(merged_labelstrings_tokenized['input_ids'].size())
# for key in merged_labelstrings_tokenized:
#     merged_labelstrings_tokenized[key] = merged_labelstrings_tokenized[key].view(num_labels, num_labelstrings[0], -1)

# tokenizer.pad_token = tokenizer.eos_token    
# tokenizer.pad_token_id = tokenizer.eos_token_id
